In [7]:
import numpy as np
import pandas as pd 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
df=pd.read_csv("tested.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)

In [49]:
df.head()
# df.isnull().sum()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q
3,0,3,male,27.0,0,0,8.6625,S
4,1,3,female,22.0,1,1,12.2875,S


In [15]:
X=df.iloc[:,1:8]
Y=df.iloc[:,0]
from sklearn.model_selection import train_test_split
[X_train,X_test,Y_train,Y_test]=train_test_split(X,Y)

In [27]:
imputer=SimpleImputer()
X_train_age=imputer.fit_transform(X_train[["Age","Fare"]])
X_test_age=imputer.transform(X_test[["Age","Fare"]])
# X_train_age=pd.DataFrame(X_train_age)
# X_train_age.isnull().sum()

0    0
1    0
dtype: int64

In [89]:
encoder=OneHotEncoder(handle_unknown="ignore")
X_train_new=encoder.fit_transform(X_train[["Sex","Embarked"]]).toarray()
X_test_new=encoder.transform(X_test[["Sex","Embarked"]]).toarray()
np.hstack((X_train[["Pclass","Age","SibSp","Parch","Fare"]].values,X_train_new))
np.hstack((X_test[["Pclass","Age","SibSp","Parch","Fare"]].values,X_test_new))

array([[ 3., 23.,  0., ...,  0.,  0.,  1.],
       [ 3., 35.,  0., ...,  0.,  1.,  0.],
       [ 2., 30.,  1., ...,  0.,  0.,  1.],
       ...,
       [ 1., 17.,  0., ...,  0.,  0.,  1.],
       [ 3., 24.,  0., ...,  0.,  1.,  0.],
       [ 2., nan,  0., ...,  1.,  0.,  0.]])

In [95]:
X_train_new2=X_train.drop(columns=["Age","Fare","Sex","Embarked"])
X_train_transformed=np.concatenate((X_train_new2,X_train_age,X_train_new),axis=1)
X_test_new2=X_test.drop(columns=["Age","Fare","Sex","Embarked"])
X_test_transformed=np.concatenate((X_test_new2,X_test_age,X_test_new),axis=1)

In [115]:
#if there would have been missing values in embarked,apply simple imputer only but mention strategy="most frequent" in the argument
#if there would have been missing values in embarked,we would make another object for it as even after filling the missing values the reault is stored in a different array
clf=DecisionTreeClassifier()
clf.fit(X_train_transformed,Y_train)
Y_pred=clf.predict(X_test_transformed)
from sklearn.metrics import accuracy_score
accuracy_score(Y_pred,Y_test)

1.0

In [121]:
import pickle
import os

# Create 'models' directory if it doesn't exist
if not os.path.exists('models'):
    os.makedirs('models')

# Saving the model
pickle.dump(clf, open("models/clf.pkl", "wb"))

# Saving the encoder
pickle.dump(encoder, open("models/encoder.pkl", "wb"))



<h2>Pipelining</h2>

In [180]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
df=pd.read_csv("tested.csv")
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)
X=df.iloc[:,1:8]
Y=df.iloc[:,0]
[X_train,X_test,Y_train,Y_test]=train_test_split(X,Y)
tnf1=ColumnTransformer([("tnf1",SimpleImputer(),[3,6])],remainder="passthrough")  #pass the columns as index as column transformer returns numpy array which doesnt have column names 
tnf2=ColumnTransformer([("tnf2",OneHotEncoder(handle_unknown="ignore"),[2,7])],remainder="passthrough")
tnf3=ColumnTransformer([("tnf3",MinMaxScaler(),slice(0,10))]) #as we are using feature selection here we need to use min max scaler,here we take 0 to 10 as the after one hot encoding 2 columns will be added for sex and 3 columns will be added for embarked and 2 columns will be dropped 
tnf4=SelectKBest(score_func=chi2,k=5)
tnf5=DecisionTreeClassifier()
pipe=Pipeline([("tnf1",tnf1),("tnf2",tnf2),("tnf3",tnf3),("tnf4",tnf4),("tnf5",tnf5)])
pipe.fit(X_train,Y_train)

ValueError: all features must be in [0, 6] or [-7, 0]

In [160]:
# pipe=make_pipeline(tnf1,tnf2,tnf3,tnf4,tnf5) #creates pipeline too,easier syntax

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'S'